# 处理丢失数据

有两种丢失数据：
- None
- np.nan(NaN)

In [1]:
import numpy as np

In [5]:
type(None) # None是python的一种对象类型

NoneType

In [7]:
type(np.nan)

float

In [9]:
np.nan-1000

nan

In [10]:
None +100

TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [11]:
1000 + "1234"

TypeError: unsupported operand type(s) for +: 'int' and 'str'

## 1. None

None是Python自带的，其类型为python object。因此，None不能参与到任何计算中。

object类型的运算要比int类型的运算慢得多  
计算不同数据类型求和时间  
%timeit np.arange(1e5,dtype=xxx).sum()

In [13]:
None + None # None不参与任何运算

TypeError: unsupported operand type(s) for +: 'NoneType' and 'NoneType'

In [14]:
"1234" + "QWER"

'1234QWER'

In [21]:
%timeit np.arange(1e5,dtype=int).sum()

123 µs ± 1.29 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [22]:
%timeit np.arange(1e5,dtype=float).sum()

159 µs ± 6.29 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [23]:
%timeit np.arange(1e5,dtype=object).sum()

9.27 ms ± 565 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## 2. np.nan（NaN）

np.nan是浮点类型，能参与到计算中。但计算的结果总是NaN。

但可以使用np.nan*()函数来计算nan，把nan直接剔除。

In [24]:
nd = np.array([1,2,np.nan,3,np.nan])
nd

array([  1.,   2.,  nan,   3.,  nan])

In [25]:
nd.sum()

nan

In [26]:
np.nansum(nd)

6.0

## 3. pandas中的None与NaN

In [27]:
import pandas as pd
from pandas import Series,DataFrame

### 1) pandas中None与np.nan都视作np.nan

创建DataFrame

In [28]:
df = DataFrame([12,2,35,7,np.nan,67,None,89],index=list("abcdefgh"),columns=["python"])
df

,python
a,12.0
b,2.0
c,35.0
d,7.0
e,NaN
f,67.0
g,NaN
h,89.0


In [29]:
df.sum() # pandas中聚合时会把nan直接剔除

python    212.0
dtype: float64

In [30]:
df.mean()

python    35.333333
dtype: float64

### 2) pandas中None与np.nan的操作

- ``isnull()``
- ``notnull()``
- ``dropna()``: 过滤丢失数据
- ``fillna()``: 填充丢失数据

In [37]:
df = DataFrame({
    "python":[120,23,np.nan,78,None],
    "java":[100,80,23,None,np.nan],
    "数学":[90,19,78,67,89],
    "H5":[78,None,np.nan,79,98]
},index=list("abcde"))
df

,H5,java,python,数学
a,78.0,100.0,120.0,90
b,NaN,80.0,23.0,19
c,NaN,23.0,NaN,78
d,79.0,NaN,78.0,67
e,98.0,NaN,NaN,89


1）判断函数

    isnull()
    notnull()

In [38]:
df.isnull()
# 输出了一个DataFrame，和原来的形状一样，值是bool

,H5,java,python,数学
a,False,False,False,False
b,True,False,False,False
c,True,False,True,False
d,False,True,False,False
e,False,True,True,False


In [39]:
df.notnull()

,H5,java,python,数学
a,True,True,True,True
b,False,True,True,True
c,False,True,False,True
d,True,False,True,True
e,True,False,False,True


In [42]:
df[df.isnull()] # 对于二维数据不能这样输出

,H5,java,python,数学
a,NaN,NaN,NaN,NaN
b,NaN,NaN,NaN,NaN
c,NaN,NaN,NaN,NaN
d,NaN,NaN,NaN,NaN
e,NaN,NaN,NaN,NaN


In [45]:
df

,H5,java,python,数学
a,78.0,100.0,120.0,90
b,NaN,80.0,23.0,19
c,NaN,23.0,NaN,78
d,79.0,NaN,78.0,67
e,98.0,NaN,NaN,89


In [49]:
cond = df.isnull().any(axis=1)
# any只要有一个为True就判定这一行或者这一列为True
# axis=0的时候，判断列上的行标中是否有True
# axis=1的时候，判断行上的列标中是否有True
cond

a    False
b     True
c     True
d     True
e     True
dtype: bool

In [50]:
df[cond]

,H5,java,python,数学
b,NaN,80.0,23.0,19
c,NaN,23.0,NaN,78
d,79.0,NaN,78.0,67
e,98.0,NaN,NaN,89


In [53]:
cond = df.isnull().any(axis=0)
cond

H5         True
java       True
python     True
数学        False
dtype: bool

In [52]:
df[cond] # 有缺失的列不允许输出

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match

推广

In [57]:
# 判断哪些大于100
df>=80 # 大于80的为True，否则为False

,H5,java,python,数学
a,False,True,True,True
b,False,True,False,False
c,False,False,False,False
d,False,False,False,False
e,True,False,False,True


In [59]:
df[df>80] # 为True地方原样输出，为False的地方输出为nan

,H5,java,python,数学
a,NaN,100.0,120.0,90.0
b,NaN,NaN,NaN,NaN
c,NaN,NaN,NaN,NaN
d,NaN,NaN,NaN,NaN
e,98.0,NaN,NaN,89.0


In [63]:
# 那些行中有大于80的
(df>80).any(axis=1)

a     True
b    False
c    False
d    False
e     True
dtype: bool

In [67]:
# 那些行的值全部大于30
df[(df>30).all(axis=1)]

,H5,java,python,数学
a,78.0,100.0,120.0,90


2) 过滤函数

In [68]:
df

,H5,java,python,数学
a,78.0,100.0,120.0,90
b,NaN,80.0,23.0,19
c,NaN,23.0,NaN,78
d,79.0,NaN,78.0,67
e,98.0,NaN,NaN,89


In [70]:
df.dropna(axis=0) 

,H5,java,python,数学
a,78.0,100.0,120.0,90


In [71]:
df.dropna(axis=1)

,数学
a,90
b,19
c,78
d,67
e,89


In [74]:
df.dropna(axis=0,how="all")

,H5,java,python,数学
a,78.0,100.0,120.0,90
b,NaN,80.0,23.0,19
c,NaN,23.0,NaN,78
d,79.0,NaN,78.0,67
e,98.0,NaN,NaN,89


dropna()用于删除缺失的那些行（或者列）

    axis默认为0，代表删除的是行，1代表删除的是列
    how 默认为"any"，代表只要有nan就删除，"all"全部为nan才删除

3）填补fillna()

In [75]:
df

,H5,java,python,数学
a,78.0,100.0,120.0,90
b,NaN,80.0,23.0,19
c,NaN,23.0,NaN,78
d,79.0,NaN,78.0,67
e,98.0,NaN,NaN,89


In [78]:
df.fillna(0) # 一般的填充

,H5,java,python,数学
a,78.0,100.0,120.0,90
b,0.0,80.0,23.0,19
c,0.0,23.0,0.0,78
d,79.0,0.0,78.0,67
e,98.0,0.0,0.0,89


向前填充与向后填充

In [80]:
df

,H5,java,python,数学
a,78.0,100.0,120.0,90
b,NaN,80.0,23.0,19
c,NaN,23.0,NaN,78
d,79.0,NaN,78.0,67
e,98.0,NaN,NaN,89


In [82]:
df.fillna(method="bfill",axis=1)
# 拿后面的数字来填充到前面缺失中，
# axis为0代表用后面的行标来填，axis为1代表用后面的列标来填

,H5,java,python,数学
a,78.0,100.0,120.0,90.0
b,80.0,80.0,23.0,19.0
c,23.0,23.0,78.0,78.0
d,79.0,78.0,78.0,67.0
e,98.0,89.0,89.0,89.0


In [85]:
df.fillna(method="ffill",axis=1)
# 拿前面的填充到后面的nan中
# axis默认为0，代表拿前面的行标填后面的nan，为1代表拿前面的列标填后面nan

,H5,java,python,数学
a,78.0,100.0,120.0,90.0
b,NaN,80.0,23.0,19.0
c,NaN,23.0,23.0,78.0
d,79.0,79.0,78.0,67.0
e,98.0,98.0,98.0,89.0


============================================

练习7：

1. 简述None与NaN的区别

2. 假设张三李四参加模拟考试，但张三因为突然想明白人生放弃了英语考试，因此记为None，请据此创建一个DataFrame,命名为ddd3

3. 老师决定根据用数学的分数填充张三的英语成绩，如何实现？
    用李四的英语成绩填充张三的英语成绩？

============================================